In [1]:
from common import *

DAY = 3

In [2]:
show_task(DAY)

<article class="day-desc"><h2>--- Day 3: Gear Ratios ---</h2><p>You and the Elf eventually reach a <a href="https://en.wikipedia.org/wiki/Gondola_lift" target="_blank">gondola lift</a> station; he says the gondola lift will take you up to the <em>water source</em>, but this is as far as he can bring you. You go inside.</p>
<p>It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.</p>
<p>"Aaah!"</p>
<p>You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.</p>
<p>The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can <em>add up all the part numbers</em> in the engine schematic, it should be easy to work out which part is missing.</p>
<p>The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently <em>any number adjacent to a symbol</em>, even diagonally, is a "part number" and should be included in your sum. (Periods (<code>.</code>) do not count as a symbol.)</p>
<p>Here is an example engine schematic:</p>
<pre><code>467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
</code></pre>
<p>In this schematic, two numbers are <em>not</em> part numbers because they are not adjacent to a symbol: <code>114</code> (top right) and <code>58</code> (middle right). Every other number is adjacent to a symbol and so <em>is</em> a part number; their sum is <code><em>4361</em></code>.</p>
<p>Of course, the actual engine schematic is much larger. <em>What is the sum of all of the part numbers in the engine schematic?</em></p>
</article>

In [30]:
lines = get_input_lines(DAY)

In [38]:
def is_touched(line, prev_line, next_line, i) -> bool:
    siblings = set()

    def check_line(current_line, positions, non_nums_only=False):
        for pos in positions:
            if pos >= 0 and pos < len(current_line):
                char = current_line[pos]
                if char == '.':
                    continue
                if non_nums_only and char in '0123456789':
                    continue
                siblings.add(char)
    
    check_line(prev_line, [i - 1, i, i + 1], True)
    check_line(next_line, [i - 1, i, i + 1], True)
    check_line(line, [i - 1, i + 1], True)
    return len(siblings) > 0

nums = []
for line_i, line in enumerate(lines):
    next_line = lines[line_i + 1] if line_i + 1 < len(lines) else ''
    prev_line = lines[line_i - 1] if line_i > 0 else ''
    touched = False
    num_str = ''
    for i, c in enumerate(line):
        if c in '0123456789':
            num_str += c
            if not touched:
                touched = is_touched(line, prev_line, next_line, i)
        elif num_str:
            if touched:
                nums.append(int(num_str))
            num_str = ''
            touched = False
    if num_str:
        if touched:
            nums.append(int(num_str))
        num_str = ''
        touched = False


num_sum = sum([int(n) for n in nums])
print(num_sum)

539713


In [39]:
send_result(DAY, 1, num_sum)

"That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two] (solved in 30:40)"

In [40]:
show_task(DAY, 2)

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.</p>
<p>You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.</p>
<p>Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.</p>
<p>The missing part wasn't the only issue - one of the gears in the engine is wrong. A <em>gear</em> is any <code>*</code> symbol that is adjacent to <em>exactly two part numbers</em>. Its <em>gear ratio</em> is the result of <span title="They're magic gears.">multiplying</span> those two numbers together.</p>
<p>This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.</p>
<p>Consider the same engine schematic again:</p>
<pre><code>467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
</code></pre>
<p>In this schematic, there are <em>two</em> gears. The first is in the top left; it has part numbers <code>467</code> and <code>35</code>, so its gear ratio is <code>16345</code>. The second gear is in the lower right; its gear ratio is <code>451490</code>. (The <code>*</code> adjacent to <code>617</code> is <em>not</em> a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces <code><em>467835</em></code>.</p>
<p><em>What is the sum of all of the gear ratios in your engine schematic?</em></p>
</article>

In [60]:
test_lines = '''
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..'''.splitlines()

In [83]:
def is_touched_gears(line, prev_line, next_line, line_i, i_start, i_end) -> set[tuple[int, int]]:
    gears = set()
    output = ''
    for i in range(i_start - 1, i_end + 1):
        if i < 0 or i >= len(prev_line):
            continue
        output += prev_line[i]
        if prev_line[i] == '*':
            gears.add((line_i - 1, i))
    
    output += ' '

    for i in range(i_start - 1, i_end + 1):
        if i < 0 or i >= len(next_line):
            continue
        output += next_line[i]
        if next_line[i] == '*':
            gears.add((line_i + 1, i))

    output += ' '

    for i in (i_start - 1, i_end):
        if i < 0 or i >= len(line):
            continue
        output += line[i]
        if line[i] == '*':
            gears.add((line_i, i))
    
    output += ' '

    # print('=== gears:', gears, line[i_start: i_end], output)
    return gears

nums_per_gear: dict[tuple[int, int], list[int]] = {}
for line_i, line in enumerate(lines):
    next_line = lines[line_i + 1] if line_i + 1 < len(lines) else ''
    prev_line = lines[line_i - 1] if line_i > 0 else ''

    num_str = ''
    start_i = None
    end_i = None
    for i, c in enumerate(line):
        if c in '0123456789':
            num_str += c
            if start_i is None:
                start_i = i
            continue
        if num_str:
            end_i = i
            gears = is_touched_gears(line, prev_line, next_line, line_i, start_i, end_i)
            # print(num_str, gears)
            for gear in gears:
                if gear not in nums_per_gear:
                    nums_per_gear[gear] = []
                nums_per_gear[gear].append(int(num_str))
            num_str = ''
            start_i = None
            end_i = None
    if num_str:
        end_i = i
        gears = is_touched_gears(line, prev_line, next_line, line_i, start_i, end_i)
        # print(num_str, gears)
        for gear in gears:
            if gear not in nums_per_gear:
                nums_per_gear[gear] = []
            nums_per_gear[gear].append(int(num_str))
        num_str = ''
        start_i = None
        end_i = None
num_sum = 0
for gear, nums in nums_per_gear.items():
    if len(nums) == 2:
        num_sum += nums[0] * nums[1]

# print(nums_per_gear)
print(num_sum)

84159075


In [84]:
send_result(DAY, 2, num_sum)

"That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 3! (solved in 36:57)"